## Задача №7

In [5]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict, Counter
import random, time

# Constants
NUM_TOPICS = 20
SAMPLE_SIZE = 2500
MAX_FEATURES = 2000
MIN_DOC_FREQ = 20
MAX_DOC_FREQ = 0.5
NUM_ITERATIONS = 80
ALPHA = 1.0
BETA = 1.0

np.random.seed(42)
random.seed(42)

dataset = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'))
all_documents = dataset.data
all_labels = dataset.target
category_names = dataset.target_names
total_docs_available = len(all_documents)
print("Total documents available:", total_docs_available)

if SAMPLE_SIZE is None or SAMPLE_SIZE >= total_docs_available:
    documents = all_documents
    labels = all_labels
else:
    indices = np.random.choice(total_docs_available, SAMPLE_SIZE, replace=False)
    documents = [all_documents[i] for i in indices]
    labels = [all_labels[i] for i in indices]

num_documents = len(documents)

text_vectorizer = CountVectorizer(stop_words='english',
                                max_features=MAX_FEATURES,
                                min_df=MIN_DOC_FREQ,
                                max_df=MAX_DOC_FREQ)
document_term_matrix = text_vectorizer.fit_transform(documents)
vocabulary = np.array(text_vectorizer.get_feature_names_out())
vocabulary_size = len(vocabulary)
print("Vocabulary size after pruning:", vocabulary_size)

sparse_matrix = document_term_matrix.tocsr()
total_tokens = int(sparse_matrix.sum())
print("Total tokens (after pruning):", total_tokens)

token_word_ids = np.empty(total_tokens, dtype=np.int32)
token_doc_ids = np.empty(total_tokens, dtype=np.int32)
position = 0
for doc_id in range(num_documents):
    doc_row = sparse_matrix[doc_id]
    for word_id, count in zip(doc_row.indices, doc_row.data):
        token_count = int(count)
        token_word_ids[position:position+token_count] = word_id
        token_doc_ids[position:position+token_count] = doc_id
        position += token_count

num_tokens = total_tokens

print("Initializing topics (K={})...".format(NUM_TOPICS))
topic_assignments = np.random.randint(0, NUM_TOPICS, size=num_tokens)
doc_topic_counts = np.zeros((num_documents, NUM_TOPICS), dtype=np.int32)
topic_word_counts = np.zeros((NUM_TOPICS, vocabulary_size), dtype=np.int32)
topic_counts = np.zeros(NUM_TOPICS, dtype=np.int32)

for i in range(num_tokens):
    topic = topic_assignments[i]
    word = token_word_ids[i]
    doc = token_doc_ids[i]
    doc_topic_counts[doc, topic] += 1
    topic_word_counts[topic, word] += 1
    topic_counts[topic] += 1

beta_sum = vocabulary_size * BETA

print("Start Gibbs sampling: N tokens =", num_tokens)
start_time = time.time()
for iteration in range(1, NUM_ITERATIONS+1):
    iteration_start_time = time.time()
    for i in range(num_tokens):
        word = token_word_ids[i]
        doc = token_doc_ids[i]
        topic = topic_assignments[i]

        doc_topic_counts[doc, topic] -= 1
        topic_word_counts[topic, word] -= 1
        topic_counts[topic] -= 1

        left = doc_topic_counts[doc] + ALPHA
        right = (topic_word_counts[:, word] + BETA) / (topic_counts + beta_sum)
        probability = left * right
        sum_prob = probability.sum()

        if sum_prob <= 0:
            probability = np.ones(NUM_TOPICS) / NUM_TOPICS
        else:
            probability = probability / sum_prob

        random_num = np.random.rand()
        cumulative_prob = np.cumsum(probability)
        new_topic = int(np.searchsorted(cumulative_prob, random_num))
        if new_topic >= NUM_TOPICS:
            new_topic = NUM_TOPICS - 1

        topic_assignments[i] = new_topic
        doc_topic_counts[doc, new_topic] += 1
        topic_word_counts[new_topic, word] += 1
        topic_counts[new_topic] += 1

    if iteration % 10 == 0 or iteration == 1:
        print(f"Iter {iteration}/{NUM_ITERATIONS} — iter time {time.time()-iteration_start_time:.1f}s — total elapsed {time.time()-start_time:.1f}s")

print("Gibbs finished; total time {:.1f}s".format(time.time()-start_time))

topic_word_dist = (topic_word_counts + BETA) / (topic_counts[:, None] + beta_sum)
doc_topic_dist = (doc_topic_counts + ALPHA).astype(float)
doc_topic_dist = doc_topic_dist / doc_topic_dist.sum(axis=1, keepdims=True)

def get_top_words(topic_id, num_words=10):
    word_indices = np.argsort(topic_word_dist[topic_id])[::-1][:num_words]
    return vocabulary[word_indices], topic_word_dist[topic_id, word_indices]

print("\nTOP-10 words per topic:")
topic_keywords = []
for topic_id in range(NUM_TOPICS):
    words, probs = get_top_words(topic_id, 10)
    topic_keywords.append(words)
    print(f"Topic {topic_id+1}: {', '.join(words)}")

dominant_topics = np.argmax(doc_topic_dist, axis=1)
topic_to_documents = defaultdict(list)
for doc_id, topic_id in enumerate(dominant_topics):
    topic_to_documents[topic_id].append(doc_id)


Total documents available: 11314
Vocabulary size after pruning: 1508
Total tokens (after pruning): 120332
Initializing topics (K=20)...
Start Gibbs sampling: N tokens = 120332
Iter 1/80 — iter time 4.0s — total elapsed 4.0s
Iter 10/80 — iter time 4.0s — total elapsed 38.3s
Iter 20/80 — iter time 3.8s — total elapsed 75.8s
Iter 30/80 — iter time 3.8s — total elapsed 114.5s
Iter 40/80 — iter time 4.1s — total elapsed 153.5s
Iter 50/80 — iter time 3.8s — total elapsed 192.0s
Iter 60/80 — iter time 3.9s — total elapsed 230.6s
Iter 70/80 — iter time 3.8s — total elapsed 268.6s
Iter 80/80 — iter time 3.9s — total elapsed 307.2s
Gibbs finished; total time 307.2s

TOP-10 words per topic:
Topic 1: edu, insurance, post, private, university, like, science, colorado, interested, make
Topic 2: think, ve, people, work, president, just, going, lot, administration, program
Topic 3: thanks, problem, windows, help, window, know, does, screen, like, hi
Topic 4: people, gun, law, like, guns, state, contro

Здесь можно предположить, что: 
Topic 1: sci.med
Topic 2: talk.politics.misc
Topic 3: comp.os.ms-windows.misc
Topic 4: talk.politics.guns?
Topic 5: talk.politics.misc?
Topic 6: sci.space
Topic 7: трудно определить
Topic 8: sci.crypt
Topic 9: misc.forsale?
Topic 10: нельзя определить
Topic 11: rec.sport.hockey
Topic 12: talk.politics.mideast
Topic 13: soc.religion.christian
Topic 14: comp.sys.ibm.pc.hardware
Topic 15: трудно определить
Topic 16: rec.autos
Topic 17: трудно определить
Topic 18: comp.windows.x
Topic 19: трудно определить
Topic 20: talk.politics.misc